# Create album

In [2]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from sklearn.preprocessing import StandardScaler
from keras.models import Model
from keras.layers import *    
import os 
import pyreadr
import numpy as np
import pyreadr
import pandas as pd

Using TensorFlow backend.


In [3]:
DF = pyreadr.read_r('/Users/mxenoc/Desktop/workspace/butterfly/data/omics.RData')
#DF = pyreadr.read_r('/home/mxenoc/workspace/butterfly/data/omics.RData')
DF = DF["DF"]

In [4]:
import butterfly.album

In [5]:
import butterfly.CNN

In [6]:
# Choose your responses, size, for the image and number of features we are predicting
response = 'plasma_l'
pixels = 40
features = 62

In [7]:
#Decide which dataset you want to predict
response = [col for col in DF if col.startswith(response)]
response.append("patientID")
response.append("trimester")    
response_df = DF[response]
#Make sure your B trimesters are properly converted to 4

In [8]:
#Get every combination of patient IDs in pairs of m's
# Function which returns subset or r length from n 
from itertools import combinations 

m = 2
IDs = response_df['patientID'].unique()
IDs = IDs.tolist()
exclude = list(combinations(IDs, m))

In [9]:
predictors = ['rna', 'serum_l', 'microb', 'immune', 'metabol', 'plasma_s']

In [10]:
predictor1 = predictors[0]
predictor2 = predictors[1]
predictor3 = predictors[2]
predictor4 = predictors[3]
predictor5 = predictors[4]
predictor6 = predictors[5]

In [11]:
album1 = butterfly.album.create_album(DF, predictor1, pixels)
album2 = butterfly.album.create_album(DF, predictor2, pixels)
album3 = butterfly.album.create_album(DF, predictor3, pixels)
album4 = butterfly.album.create_album(DF, predictor4, pixels)
album5 = butterfly.album.create_album(DF, predictor5, pixels)
album6 = butterfly.album.create_album(DF, predictor6, pixels)

In [13]:
yy = response_df.drop(['patientID', 'trimester'], axis =1 ).values
prediction = []
observed = []

In [ ]:
for cv in range(len(exclude)):
    #Create your calibration and validation datasets
    pt1ex = response_df.index[response_df['patientID'] == exclude[cv][0]].tolist()
    pt2ex = response_df.index[response_df['patientID'] == exclude[cv][1]].tolist()
    ptex = pt1ex+pt2ex

    #Divide in calibration and validation
    X_c1 = np.delete(album1, ptex, 0)
    X_c1 = X_c1.reshape((X_c1.shape[0], X_c1.shape[1], pixels))
    
    X_c2 = np.delete(album2, ptex, 0)
    X_c2 = X_c2.reshape((X_c2.shape[0], X_c2.shape[1], pixels))

    X_c3 = np.delete(album3, ptex, 0)
    X_c3 = X_c3.reshape((X_c3.shape[0], X_c3.shape[1], pixels))

    X_c4 = np.delete(album4, ptex, 0)
    X_c4 = X_c4.reshape((X_c4.shape[0], X_c4.shape[1], pixels))

    X_c5 = np.delete(album5, ptex, 0)
    X_c5 = X_c5.reshape((X_c5.shape[0], X_c5.shape[1], pixels))

    X_c6 = np.delete(album6, ptex, 0)
    X_c6 = X_c6.reshape((X_c6.shape[0], X_c6.shape[1], pixels))
    
    y_c = np.delete(yy, ptex, 0)
    y_c = pd.DataFrame(StandardScaler().fit_transform(y_c))
    
    X_c = []

    for i in range(60):
        X_c.append(np.array((X_c1[i], X_c2[i], X_c3[i], X_c4[i], X_c5[i], X_c6[i]), dtype=float))
    
    X_c = np.array(X_c)
    X_c = X_c.reshape((X_c.shape[0], pixels, pixels, X_c.shape[1]))

    X_v1 = np.asarray([album1[i]  for i in ptex])
    X_v1 = X_v1.reshape((X_v1.shape[0], X_v1.shape[1], pixels))

    X_v2 = np.asarray([album2[i]  for i in ptex])
    X_v2 = X_v2.reshape((X_v2.shape[0], X_v2.shape[1], pixels))

    X_v3 = np.asarray([album3[i]  for i in ptex])
    X_v3 = X_v3.reshape((X_v3.shape[0], X_v3.shape[1], pixels))

    X_v4 = np.asarray([album4[i]  for i in ptex])
    X_v4 = X_v4.reshape((X_v4.shape[0], X_v4.shape[1], pixels))

    X_v5 = np.asarray([album5[i]  for i in ptex])
    X_v5 = X_v5.reshape((X_v5.shape[0], X_v5.shape[1], pixels))

    X_v6 = np.asarray([album6[i]  for i in ptex])
    X_v6 = X_v6.reshape((X_v6.shape[0], X_v6.shape[1], pixels))    
    
    y_v = np.asarray([yy[i] for i in ptex])
    y_v = pd.DataFrame(StandardScaler().fit_transform(y_v))
    
    X_v = []

    for i in range(8):
        X_v.append(np.array((X_v1[i], X_v2[i], X_v3[i], X_v4[i], X_v5[i], X_v6[i]), dtype=float))
    
    X_v = np.array(X_v)
    X_v = X_v.reshape((X_v.shape[0], pixels, pixels, X_v.shape[1]))

    #Create your CNN
    model = Sequential()
    model.add(Conv2D(32,(3,3), input_shape=(pixels, pixels,6)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Flatten())
    model.add(Dense(64))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))
    model.add(Dense(features))
    model.add(Activation( 'sigmoid'))
    model.compile(optimizer='adam', loss='mse')

    model.fit(X_c, y_c, epochs=300, verbose=0)

    # demonstrate prediction
    y_pred = model.predict(X_v, verbose = 0)
    y_pred = pd.DataFrame(y_pred)
    prediction.append(y_pred)
    observed.append(y_v)

In [27]:
#from joblib import Parallel, delayed
#import multiprocessing

#num_cores = multiprocessing.cpu_count()

#results = Parallel(n_jobs=num_cores)(delayed(processInput)(i) for i in inputs)

In [28]:
pred = pd.concat(prediction)

In [29]:
obs = pd.concat(observed)

In [30]:
correl = pred.corrwith(obs, axis = 0) 

In [183]:
#correl = bars_p[number].corrwith(bars_o[number], axis = 0) 

In [32]:
correl

0     0.402697
1     0.427385
2     0.367852
3     0.389154
4     0.409281
        ...   
57    0.387163
58    0.363417
59    0.425253
60    0.442869
61    0.235089
Length: 62, dtype: float64

In [33]:
correl[correl < 0] = 0

In [36]:
max(correl)

0.5154415273036915

In [37]:
min(correl)

0.23508928964231607

In [34]:
correl

0     0.402697
1     0.427385
2     0.367852
3     0.389154
4     0.409281
        ...   
57    0.387163
58    0.363417
59    0.425253
60    0.442869
61    0.235089
Length: 62, dtype: float64

In [35]:
np.mean(correl)

0.37873824750989293